### Calculate coefficient of variation as per Glüer et al. (1995)

Author: Simone Poncioni, MSB, ARTORG Center for Biomedical Engineering Research, University of Bern, Switzerland

Date: 07.2024

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import yaml
from collections import Counter, defaultdict

import matplotlib.pyplot as plt

In [ ]:
def calculate_cv_gluer(infile: pd.DataFrame, outfile: dict, variable: str):
    """Function to calculate the coefficient of variation as per Glüer et al. 1995

    Args:
        infile (pd.DataFrame): DataFrame containing the imported yaml file with the simulation grouping information
        outfile (dict): simulation results file
        variable (str): variable to analyse (e.g. 'Stiffness', 'yield_force', etc.)
    """
    def dofs(results):
        '''
        Equation (7), calculating the degrees of fredom df_j of the measurements in the individuals
        '''
        counter = Counter()
        for value in results.values():
            key_part = value[0].split('/')[1]
            counter[key_part] += 1
        degrees_of_freedom = sum(count - 1 for count in counter.values())
        return degrees_of_freedom

    def precision_error(results):
        '''
        Equation (6), calculating the generic standard deviation sd of the measurements
        '''
        values_sorted = defaultdict(list)
        for _, value in results.items():
            key_part = value[0].split('/')[1]
            values_sorted[key_part].append(value[1])
        
        sum_tot = 0
        total_elements = 0
        df = 0

        # Calculating the internal mean, sum of squared differences, and degrees of freedom
        for key, values in values_sorted.items():
            internal_mean = sum(values) / len(values)
            total_elements += len(values)
            df += len(values) - 1  # Adding to degrees of freedom
            
            for xij in values:
                sum_tot += (xij - internal_mean) ** 2

        # Dividing by degrees of freedom
        variance = sum_tot / df
        sd = variance ** 0.5

        return sd, values_sorted

    def coefficient_of_variation(sd, values_sorted):
        '''
        Equation (5), calculating the coefficient of variation on a percentage basis
        '''
        HUNDRED = 100
        sum_means = 0
        m = len(values_sorted)

        for _, values in values_sorted.items():
            internal_mean = sum(values) / len(values)
            sum_means += internal_mean
        
        mean_of_means = sum_means / m
        cv_sd = (sd / mean_of_means) * HUNDRED
        return cv_sd

    out_samples = outfile['Sample'].values
    out_samples = [str(sample) for sample in out_samples]
    results_dict = {}
    for sample in out_samples:
        variable_res = outfile[outfile['Sample'] == sample][str(variable)]
        # add key 'variable' to results_dict (which has sample as key, infile['simulations']['folder_id'][sample] as value 0, and 'variable' as value 1)
        results_dict[sample] = [infile['simulations']['folder_id'][sample], variable_res.values[0]]

    # split results_dict into two dictionaries: results_r and results_t
    # ->>> results_r contains infile['simulations']['folder_id'][sample][-1] == 'R'
    # ->>> results_t contains infile['simulations']['folder_id'][sample][-1] == 'T'

    results_r = {}
    results_t = {}
    for key, value in results_dict.items():
        if value[0][-1] == 'R':
            results_r[key] = value
        elif value[0][-1] == 'T':
            results_t[key] = value
            
    property_r = [value[1] for value in results_r.values()]
    property_t = [value[1] for value in results_t.values()]

    property = [property_r, property_t]
    results = [results_r, results_t]
    site = ['RADIUS', 'TIBIA']

    df1 = pd.DataFrame()
    # as per Glüer et al. 1995
    for res, stiff, _site in zip(results, property, site):
        m = len(stiff)
        df = dofs(res)
        sd, values_sorted = precision_error(res)
        cv_sd = coefficient_of_variation(sd, values_sorted)
        # append to df
        df_temp = pd.DataFrame({'Variable': variable, 'm': m, 'df': df, 'sd': sd, 'cv_sd': cv_sd}, index=[_site])
        df1 = pd.concat([df1, df_temp])

    df1.index.name = 'Site'
    df1.reset_index(inplace=True)
    df1.set_index(['Site', 'Variable'], inplace=True)
    df1.reset_index(inplace=True)  # Reset index to make 'Site' and 'Variable' regular columns

    return df1


def pivot_dataframe(df: pd.DataFrame):
    df_res_pivot = df.pivot(index='Variable', columns='Site')
    df_res_pivot.columns = ['_'.join(col).strip() for col in df_res_pivot.columns.values]
    new_column_order = ['m_RADIUS', 'df_RADIUS', 'sd_RADIUS', 'cv_sd_RADIUS', 'm_TIBIA', 'df_TIBIA', 'sd_TIBIA', 'cv_sd_TIBIA']
    df_res_pivot = df_res_pivot[new_column_order]
    return df_res_pivot

In [ ]:
# Load the results
in_path = Path('summaries/simulations-repro.yaml')
with open(in_path, 'r') as f:
    infile = yaml.safe_load(f)

# * Indermaur et al. (in preparation, 2024)
# outfile_path = Path('summaries/REPRO_mech_param_FEA_noReg__V_00_noReg_FZ_MAX_sphere_changed.csv')
# variables_to_analyse = ['Stiffness', 'yield_force']

# * Ours (Poncioni et al., in preparation, 2024)
outfile_path = Path('summaries/06_repro_paper_data_summary.csv')
# variables_to_analyse = ['stiffness_1D_FZ_MAX', 'yield_force_FZ_MAX', 'yield_disp_FZ_MAX', 'max_force_FZ_MAX', 'disp_at_max_force_FZ_MAX', 'Volume', 'tb_bv', 'ct_bv']
variables_to_analyse = ['stiffness_1D_FZ_MAX', 'yield_force_FZ_MAX']
outfile = pd.read_csv(outfile_path, delimiter=',')

In [ ]:
# Analyze the results as per Glüer et al. 1995
df_results = pd.DataFrame()

for var in variables_to_analyse:
    df_res = calculate_cv_gluer(infile, outfile, var)
    df_res_pivot = pivot_dataframe(df_res)
    df_results = pd.concat([df_results, df_res_pivot])

print(df_results.to_string())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the results
in_path = Path('summaries/simulations-repro.yaml')
with open(in_path, 'r') as f:
    infile = yaml.safe_load(f)


outfiles = [Path('summaries/REPRO_mech_param_FEA_noReg__V_00_noReg_FZ_MAX_sphere_changed.csv'), Path('summaries/03_repro_vtu_data_summary_with_volumes.csv')]
variables_to_analyse_list = [['Stiffness', 'yield_force', 'yield_disp', 'max_force', 'disp_max_force'], ['stiffness_1D_FZ_MAX', 'yield_force_FZ_MAX', 'yield_disp_FZ_MAX', 'max_force_FZ_MAX', 'disp_at_max_force_FZ_MAX']]

df_results = pd.DataFrame()
for outfile, variables_to_analyse in zip(outfiles, variables_to_analyse_list):
    
    outfile_df = pd.read_csv(outfile, delimiter=',')
    for var in variables_to_analyse:
        df_res = calculate_cv_gluer(infile, outfile_df, var)
        # Pivot the DataFrame
        df_res_pivot = pivot_dataframe(df_res)
        
        df_results = pd.concat([df_results, df_res_pivot])
    print(df_results.to_string())
    if outfile == outfiles[0]:
        df1 = df_results.copy()
    else:
        df2 = df_results.copy()

# df_tot = pd.concat([df1, df2], axis=0)
df_tot = df2
# Assuming df1 and df2 are already loaded

# Normalize column names for comparison
# Map df1 columns to df2 columns based on the order specified in the question
columns_mapping = {
    'Stiffness': 'stiffness_1D_FZ_MAX',
    'yield_force': 'yield_force_FZ_MAX',
    'yield_disp': 'yield_disp_FZ_MAX',
    'max_force': 'max_force_FZ_MAX',
    'disp_max_force': 'disp_at_max_force_FZ_MAX'
}

# Prepare comparison data
comparison_data = {
    'Variable': [],
    'cv_sd_RADIUS_df1': [],
    'cv_sd_TIBIA_df1': [],
    'cv_sd_RADIUS_df2': [],
    'cv_sd_TIBIA_df2': []
}

print(f'Complete data:\n\n{df_tot.to_string()}')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('/home/simoneponcioni/Documents/00_GENERAL/pos_monitor.mplstyle')

# get index of the dataframe
index = df_tot.index

# Mapping of properties
columns_mapping = {
    'Stiffness': 'stiffness_1D_FZ_MAX',
    'yield_force': 'yield_force_FZ_MAX',
    'yield_disp': 'yield_disp_FZ_MAX',
    'max_force': 'max_force_FZ_MAX',
    'disp_max_force': 'disp_at_max_force_FZ_MAX'
}

# Create a new DataFrame with both original and mapped values
properties = list(columns_mapping.keys())
mapped_properties = list(columns_mapping.values())

# Extract cv_sd_RADIUS values
original_values = df_tot.loc[properties, 'cv_sd_RADIUS']
mapped_values = df_tot.loc[mapped_properties, 'cv_sd_RADIUS']

# Create a new DataFrame to store the results
plot_data = pd.DataFrame({
    'Property': properties + mapped_properties,
    'cv_sd_RADIUS': list(original_values) + list(mapped_values)
})

# Create a new DataFrame with both original and mapped values
plot_data = []
for orig, mapped in columns_mapping.items():
    plot_data.append({
        'Property': orig,
        'cv_sd_RADIUS': abs(df_tot.loc[orig, 'cv_sd_RADIUS']),
        'Type': 'Original'
    })
    plot_data.append({
        'Property': mapped,
        'cv_sd_RADIUS': abs(df_tot.loc[mapped, 'cv_sd_RADIUS']),
        'Type': 'Mapped'
    })

plot_data = pd.DataFrame(plot_data)

plt.figure(figsize=(6, 6))
bar_width = 0.5
indices = np.arange(len(columns_mapping))
positions = np.array([2 * i for i in range(len(columns_mapping))])

original_bars = plt.bar(positions, plot_data[plot_data['Type'] == 'Original']['cv_sd_RADIUS'], bar_width, label='Indermaur et al. (2021)')
spline_bars = plt.bar(positions + bar_width, plot_data[plot_data['Type'] == 'Mapped']['cv_sd_RADIUS'], bar_width, label='Ours')

# Add labels
plt.ylabel('$CV_{SD}$ Radius')
plt.title('Comparison of CV - Radius', weight='bold')
middle_positions = positions + bar_width // 2
plt.xticks(ticks=middle_positions, labels=columns_mapping.keys(), rotation=45)
plt.legend()

# Show plot
plt.tight_layout()
plt.show()


In [ ]:
# get index of the dataframe
index = df_tot.index

# Mapping of properties
columns_mapping = {
    'Stiffness': 'stiffness_1D_FZ_MAX',
    'yield_force': 'yield_force_FZ_MAX',
    'yield_disp': 'yield_disp_FZ_MAX',
    'max_force': 'max_force_FZ_MAX',
    'disp_max_force': 'disp_at_max_force_FZ_MAX'
}

# Create a new DataFrame with both original and mapped values
properties = list(columns_mapping.keys())
mapped_properties = list(columns_mapping.values())

# Extract cv_sd_TIBIA values
original_values = df_tot.loc[properties, 'cv_sd_TIBIA']
mapped_values = df_tot.loc[mapped_properties, 'cv_sd_TIBIA']

# Create a new DataFrame to store the results
plot_data = pd.DataFrame({
    'Property': properties + mapped_properties,
    'cv_sd_TIBIA': list(original_values) + list(mapped_values)
})

# Create a new DataFrame with both original and mapped values
plot_data = []
for orig, mapped in columns_mapping.items():
    plot_data.append({
        'Property': orig,
        'cv_sd_TIBIA': abs(df_tot.loc[orig, 'cv_sd_TIBIA']),
        'Type': 'Original'
    })
    plot_data.append({
        'Property': mapped,
        'cv_sd_TIBIA': abs(df_tot.loc[mapped, 'cv_sd_TIBIA']),
        'Type': 'Mapped'
    })

plot_data = pd.DataFrame(plot_data)

plt.figure(figsize=(6, 6))
bar_width = 0.5
indices = np.arange(len(columns_mapping))
positions = np.array([2 * i for i in range(len(columns_mapping))])

original_bars = plt.bar(positions, plot_data[plot_data['Type'] == 'Original']['cv_sd_TIBIA'], bar_width, label='Indermaur et al. (2021)')
spline_bars = plt.bar(positions + bar_width, plot_data[plot_data['Type'] == 'Mapped']['cv_sd_TIBIA'], bar_width, label='Ours')

# Add labels
plt.ylabel('$CV_{SD}$ Tibia')
plt.title('Comparison of CV - Tibia', weight='bold')
middle_positions = positions + bar_width // 2
plt.xticks(ticks=middle_positions, labels=columns_mapping.keys(), rotation=45)
plt.legend()

# Show plot
plt.tight_layout()
plt.show()
